# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Sytems Biology Markup Language (SBML) format.

Using COBRApy, one can analyse the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change the environmental conditions.

Phenotype prediction can be simulated with several flux analysis methods implemented in COBRApy. These include Flux Balance Anlysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a simple and straightforward process. One can simulate single or double knock outs using one of the flux analysis methods mentioned above.

## Instalation


### Requirements
The following requirements need to be installed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK solver is used by default but CPLEX is prefered


### How to install COBRApy?
```
pip install cobra
```

# Exercise 5 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _iMM904_ model which contains the whole metabolism of _S. cerevisiae_ -> file: ../data/iMM904.xml

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Loading the model with COBRApy;
- Perform a FBA simulation using an aerobic/anaerobic medium;
- Perform reaction and gene deletions
- Perform other flux analysis methods, such as pFBA, FVA, ROOM and MOMA;
- Analyzing the model essential 

In [28]:
# importing cobra
import cobra

# Loading a model
model_path = '../data/iMM904.xml'
model = cobra.io.read_sbml_model(model_path)

model

Name,iMM904
Memory address,0x019aa2f93908
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


### Phenotype Prediction

#### Flux Balance Analysis (FBA) - Aerobiose

In [29]:
#check model exchanges
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

Episterol ester  yeast specific exchange -> (0.0, 999999.0)
Episterol exchange -> (0.0, 999999.0)
Ergosterol exchange -> (0.0, 999999.0)
Ergosterol ester  yeast specific exchange -> (0.0, 999999.0)
1 3 beta D Glucan exchange -> (0.0, 999999.0)
Ethanolamine exchange -> (0.0, 999999.0)
2 Hydroxybutyrate exchange -> (0.0, 999999.0)
Ethanol exchange -> (0.0, 999999.0)
Fe2+ exchange -> (-999999.0, 999999.0)
Fecosterol exchange -> (0.0, 999999.0)
2 methylbutyl acetate exchange -> (0.0, 999999.0)
2 methylbutyraldehyde exchange -> (0.0, 999999.0)
2 methyl 1 butanol exchange -> (0.0, 999999.0)
2 methylpropanal exchange -> (0.0, 999999.0)
2 phenylethanol exchange -> (0.0, 999999.0)
Fecosterol ester  yeast specific exchange -> (0.0, 999999.0)
FMN exchange -> (0.0, 999999.0)
3 Carboxy 3 hydroxy 4 methylpentanoate exchange -> (0.0, 999999.0)
3 Methylbutanal exchange -> (0.0, 999999.0)
Formate exchange -> (0.0, 999999.0)
D-Fructose exchange -> (0.0, 999999.0)
3mop exchange -> (0.0, 999999.0)
4-Amino

In [30]:
#Perform a FBA with aerobic conditions
aerobic_solution = model.optimize()
aerobic_solution

,fluxes,reduced_costs
CITtcp,0.000000,-3.469447e-18
13BGH,0.000000,-4.751566e-02
13BGHe,0.000000,0.000000e+00
13GS,0.326670,0.000000e+00
16GS,0.000000,-9.503132e-02
...,...,...
PYDXO,0.000000,0.000000e+00
PYK,17.721990,0.000000e+00
PYNP2r,0.000000,-0.000000e+00
PYR5CDm,0.000000,-2.969729e-02


In [31]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


#### Flux Balance Analysis (FBA) - Anaerobiose

In [32]:
#setting the environmental conditions to replicate an anaerobic medium
o2 = model.reactions.get_by_id('EX_o2_e')
o2.bounds = (0,1000)
o2

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x19aa5b10188
Stoichiometry,o2_e --> O2 O2 -->
GPR,
Lower bound,0
Upper bound,1000


In [33]:
#Perform a FBA with anaerobic conditions
anaerobic_solution = model.optimize()
anaerobic_solution

,fluxes,reduced_costs
CITtcp,0.0,-0.0
13BGH,0.0,0.0
13BGHe,0.0,0.0
13GS,0.0,0.0
16GS,0.0,0.0
...,...,...
PYDXO,0.0,0.0
PYK,1.0,0.0
PYNP2r,0.0,0.0
PYR5CDm,0.0,0.0


In [34]:
#setting the environmental conditions to replicate an aerobic medium
o2 = model.reactions.get_by_id('EX_o2_e')
o2.bounds = (-2,1000)
o2

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x19aa5b10188
Stoichiometry,o2_e <=> O2 O2 <=>
GPR,
Lower bound,-2
Upper bound,1000


#### Flux Variability Analysis (FVA)

In [35]:
#performing fva simulation
fva_solution = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
fva_solution

,minimum,maximum
CITtcp,0.000000,0.000000
13BGH,0.000000,10.823974
13BGHe,0.000000,0.000000
13GS,0.032667,10.856641
16GS,0.000000,0.000000
...,...,...
PYDXO,0.000000,0.000000
PYK,0.000000,41.459171
PYNP2r,-10.823974,10.823974
PYR5CDm,0.000000,9.426427


In [36]:
# maximum theoretical production rates of Acetate (EX_ac_e) and Formate (EX_for_e)

print(f'Theoretical production rate interval for Acetate:', fva_solution.loc['EX_ac_e', 'minimum'], 'to', fva_solution.loc['EX_ac_e', 'maximum'], 'mmol/gDW/h')
print(f'Theoretical production rate interval of Formate:', fva_solution.loc['EX_for_e', 'minimum'], 'to', fva_solution.loc['EX_for_e', 'maximum'], 'mmol/gDW/h')

Theoretical production rate interval for Acetate: 0.0 to 12.536161700800076 mmol/gDW/h
Theoretical production rate interval of Formate: 0.0 to 2.6299919299464114 mmol/gDW/h


#### Parsimonious Flux Balance Analysis (pFBA)

In [37]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution

,fluxes,reduced_costs
CITtcp,0.000000,-2.0
13BGH,0.000000,444.0
13BGHe,0.000000,-2.0
13GS,0.326670,-2.0
16GS,0.000000,-2.0
...,...,...
PYDXO,0.000000,-2.0
PYK,17.721990,-2.0
PYNP2r,0.000000,-2.0
PYR5CDm,0.000000,270.0


### Simulating Deletions

#### Knock outs

In [40]:
#knock out the ENO reaction
with model:
    model.reactions.ENO.knock_out()
    fba_solution = model.optimize()
    print('ENO mutant growth rate: ', fba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('ENO flux rate: ', fba_solution.fluxes['ENO'])

ENO mutant growth rate:  0.0516867210336672
ENO flux rate:  0.0


In [ ]:
#knock out the ACS gene
with model:
    model.genes.YLR153C.knock_out()
    fba_solution = model.optimize()
    print('ACS mutant growth rate: ', fba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('ACS flux rate: ', fba_solution.fluxes['ACS'])

In [ ]:
#knock out the PGM GPRs
with model:
    model.genes.YKL152C.knock_out()
    model.genes.YDL021W.knock_out()
    model.genes.YOL056W.knock_out()
    fba_solution = model.optimize()
    print('PGM mutant growth rate: ', fba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('PGM flux rate: ', fba_solution.fluxes['PGM'])

In [ ]:
#all knock outs at once
with model:
    fba_solution = model.optimize()
    model.reactions.ENO.knock_out()
    model.genes.YLR153C.knock_out()
    model.genes.YKL152C.knock_out()
    model.genes.YDL021W.knock_out()
    model.genes.YOL056W.knock_out()
    mutant_fba = model.optimize()
    print('Wild type growth rate: ', fba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('Mutant growth rate: ', mutant_fba.fluxes['BIOMASS_SC5_notrace'])

#### MOMA and ROOM

In [ ]:
with model:
    fba_solution = model.optimize()
    model.reactions.ENO.knock_out()
    model.genes.YLR153C.knock_out()
    model.genes.YKL152C.knock_out()
    model.genes.YDL021W.knock_out()
    model.genes.YOL056W.knock_out()
    moma_result = cobra.flux_analysis.moma(model, fba_solution)
    print('MOMA Result: ', moma_result)

In [ ]:
with model:
    fba_solution = model.optimize()
    model.reactions.ENO.knock_out()
    model.genes.YLR153C.knock_out()
    model.genes.YKL152C.knock_out()
    model.genes.YDL021W.knock_out()
    model.genes.YOL056W.knock_out()
    room_result = cobra.flux_analysis.room(model, fba_solution)
    print('ROOM Result: ', room_result)

#### Single Delections

In [ ]:
#single reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
double_reaction_deletion_results

In [ ]:
#double gene deletion
double_gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
double_gene_deletion_results

### Production envelopes

In [ ]:
#obtain a plot of a production envelope using glucose as carbon source and optimizing Formate production
prod_env = cobra.flux_analysis.production_envelope(model, ["BIOMASS_SC5_notrace"], objective="EX_for_e", carbon_sources="EX_glc__D_e")
prod_env.plot(kind='line', x='BIOMASS_SC5_notrace', y='carbon_yield_maximum')